#   Avaliação de hipótese 1:

Aqui vamos avaliar o impacto da "brand" para o nosso modelo.

A razão de tal análize serve para entendermos melhor qual tratamento deveremos dar para nossos dados 632682 nulos, um número significativo dentro de nosso dataset. Para isso vamos separa-los em 2 datasets, um contendo apenas os valores nulos/missing e um outro dataset contendo todos as colinas preenchidas. Após essa etápa vamos gerar modelos probatórios para  um teste a/b, onde poderemos analisar o melhor caminho para o tratamento desses valores nulos.


In [1]:
import pandas as pd
import random 
import numpy as np
import re
from scipy.sparse import hstack

In [2]:
from pre_processamento import pre_processamento

In [3]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree

In [6]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error, r2_score

In [7]:
train = pd.read_csv(r"C:\Users\angel\Desktop\bootcamp\train.tsv", sep='\t')
test = pd.read_csv(r"C:\Users\angel\Desktop\bootcamp\test.tsv", sep='\t')

In [8]:
train,test = pre_processamento(train,test)

In [9]:
train.head()

,name,item_condition_id,brand_name,price,shipping,item_description,stock,gen_cat,sub1_cat,sub2_cat,datetime_month,datetime_year
0,MLB Cincinnati Reds T Shirt Size XL,3,NaN,10.0,1,No description yet,27,Men,Tops,T-shirts,6.0,2018.0
1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,15,Electronics,Computers & Tablets,Components & Parts,3.0,2018.0
2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,14,Women,Tops & Blouses,Blouse,10.0,2018.0
3,Leather Horse Statues,1,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,1,Home,Home Décor,Home Décor Accents,3.0,2018.0
4,24K GOLD plated rose,1,NaN,44.0,0,Complete with certificate of authenticity,13,Women,Jewelry,Necklaces,5.0,2018.0


In [24]:
def text_preprocess(phrase):
    
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    phrase = re.sub('https?://\S+| www.\S+', '', phrase) #Remoção de qualquer site que possa ter no nosso dataset
    phrase  = re.sub('[rm]', '', phrase) #exclusao do "[rm]" usado pelo kaggle para substituir o preço que aparecia na coluna name e description 
    phrase = re.sub("[^a-zA-Z0-9' \n\.]", '', phrase)  #excluindo tudo o que não for letra e numero.
    
    phrase  = re.sub('^no description yet$', '', phrase)
    
    phrase = re.sub(' +', ' ', phrase) #exclusão de espaços extras, ex: "eu    vou" -> "eu vou"

    phrase = phrase.lower() #passando para letra minúscula.

   
    return phrase

In [11]:
train["name"] = train["name"].apply(lambda x: text_preprocess(x)) #limpeza do texto na coluna name

In [12]:
train["item_description"] = train["item_description"].astype(str) 

In [13]:
train["item_description"] = train["item_description"].apply(lambda x: text_preprocess(x)) #limpeza do texto na coluna de descrição

In [14]:
vec = TfidfVectorizer(stop_words='english', ngram_range=(1,1), max_features=50000) #remoção de stopword e max features criadas em 5000


In [ ]:
rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1)

#   Modelo com brands 


In [43]:
Train_with_brand = train.dropna(subset="brand_name") #Dropando todas as linhas nulas  em brand,nesse modelo só teremos valores não nulos para essa coluna

In [44]:
X1 = Train_with_brand.drop("price", axis = 1)
y1 = Train_with_brand["price"]

In [45]:
X1train, X1val, y1train, y1val = train_test_split(X1, y1, test_size=0.4, random_state=10) #split 60/40 

In [46]:
X1train.head()

,name,item_condition_id,brand_name,shipping,item_description,stock,gen_cat,sub1_cat,sub2_cat,datetime_month,datetime_year
926821,bke,4,Buckle,0,bke denim jeans size 2931 12. has a small hole...,6,Women,Jeans,Boot Cut,11.0,2018.0
1198958,new vs socks,1,Victoria's Secret,0,2 pairs,27,Women,Athletic Apparel,Socks,9.0,2018.0
902986,lululemon shirt bundle size 6,3,Lululemon,0,great condition both shirts for one price size 6,38,Women,Athletic Apparel,Shirts & Tops,6.0,2018.0
585065,10 packs my little pony figures toys,1,My Little Pony,1,10 packs my little pony figures random new in ...,3,Kids,Toys,Dolls & Accessories,4.0,2018.0
1188702,vintage gucci crossbody,3,Gucci,0,vintage no serial number needs a little tlc as...,11,Women,Women's Handbags,Messenger & Crossbody,9.0,2018.0


In [47]:

vce_xtrain_name_1 = vec.fit_transform(X1train["name"])
vce_xtrain_descrip_1 = vec.fit_transform(X1train["item_description"])

In [50]:
ohe = OneHotEncoder(handle_unknown="ignore")

#aplicação dummy nas colunas categóricas.

ohe1 = ohe.fit_transform(X1train[["item_condition_id",   
                                         "shipping",
                                         "brand_name",
                                         "gen_cat",
                                         "sub1_cat",
                                         "sub2_cat", 
                                         "datetime_month"]])

In [52]:
x1_train = hstack([                                #função para juntar as matrizes criadas
                  (vce_xtrain_name_1),
                  (vce_xtrain_descrip_1),
                  (ohe1)  
                        ])

In [53]:
x1_train

<509595x105000 sparse matrix of type '<class 'numpy.float64'>'
	with 12097132 stored elements in Compressed Sparse Row format>

In [54]:
model1 = rfr.fit(x1_train[:150000], y1train[:150000])  #devido a problemas de memorias, limitarei em 50000 linhas para um primeiro teste.

In [ ]:
#Aplicando transformações nos dados de validação


vce_xval_name_1 = vec.transform(X1val["name"])
vce_xval_descrip_1 = vec.transform(X1val["item_description"])

ohe_val_condition_1 = ohe.transform(X1val[["item_condition_id",   
                                          "shipping",
                                          "brand_name",
                                          "gen_cat",
                                          "sub1_cat",
                                          "sub2_cat", 
                                          "datetime_month"]])

x_val = hstack([(vce_xval_name_1),
                (vce_xval_descrip_1), 
                (ohe_val_condition_1)])



In [ ]:
ohe_val_condition_1

<339730x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2377353 stored elements in Compressed Sparse Row format>

# Dataset com Brand nulas

In [ ]:
train_No_brand = train.loc[train.brand_name.isnull()] #Separando todas as linhas nulas  em brand,nesse modelo só teremos valores nulos para essa coluna

In [ ]:
X2 = train_No_brand.drop("price", axis = 1)
y2 = train_No_brand["price"]

In [ ]:
X2train, X2val, y2train, y2val = train_test_split(X2, y2, test_size=0.4, random_state=10)

In [ ]:
vce_xtrain_name_2 = vec.fit_transform(X2train["name"])
vce_xtrain_descrip_2 = vec.fit_transform(X2train["item_description"])
ohe = OneHotEncoder(handle_unknown="ignore")

#aplicação dummy nas colunas categóricas.

ohe2 = ohe.fit_transform(X2train[["item_condition_id",   
                                         "shipping",
                                         "gen_cat",
                                         "sub1_cat",
                                         "sub2_cat", 
                                         "datetime_month"]])

In [ ]:
x2_train = hstack([
                  (vce_xtrain_name_2),
                  (vce_xtrain_descrip_2),
                  (ohe2)  
                        ])

In [ ]:
x2_train

<379401x10974 sparse matrix of type '<class 'numpy.float64'>'
	with 8902573 stored elements in Compressed Sparse Row format>

In [ ]:
rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1)

model2 = rfr.fit(x2_train[:50000], y2train[:50000])

In [ ]:

vce_xval_name_2 = vec.transform(X1val["name"])
vce_xval_descrip_2 = vec.transform(X1val["item_description"])

ohe_val_condition_2 = ohe.transform(X1val[["item_condition_id",   
                                          "shipping",
                                          "gen_cat",
                                          "sub1_cat",
                                          "sub2_cat", 
                                          "datetime_month"]])

x_val2 = hstack([(vce_xval_name_2),
                (vce_xval_descrip_2), 
                (ohe_val_condition_2)])


In [ ]:
x_val2

<339730x10974 sparse matrix of type '<class 'numpy.float64'>'
	with 7053612 stored elements in Compressed Sparse Row format>

#   Resultados

In [ ]:
p1 = model1.predict(x_val[:150000])

In [ ]:
print("---------------Modelo com brand----------------- ")


print('RMSLE: %2f' % mean_squared_log_error(y1val[:150000], y1train[:150000]))
print('RMSE: %2f' % (mean_squared_error(y1val[:150000], y1train[:150000])))

---------------Modelo com brand----------------- 
RMSLE: 1.129915
RMSE: 4070.070590


In [ ]:
p2 = model2.predict(x_val2[:150000])

In [ ]:
print("---------------Modelo sem brand----------------- ")


print('RMSLE: %2f' % mean_squared_log_error(y2val[:50000], y2train[:50000]))
print('RMSE: %2f' % (mean_squared_error(y2val[:50000], y2train[:50000])))

---------------Modelo sem brand----------------- 
RMSLE: 0.977183
RMSE: 1540.072510
